## Create Example - Rasterize Single Year of Fire Data

## Create Annually or Group_Year Fire Rasters


In [1]:
import geopandas as gpd
import rasterio
from rasterio import features
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from tsraster.prep import poly_rasterizer_year_group


In [2]:
# run individual date range
shp_fn = r'../Data/Actual/Fires/Inputs/fire17_1.shp'

poly_rasterizer_year_group(poly = shp_fn, raster_exmpl = r'../Data/Examples/aet-198403.tif',
                raster_path_prefix = r'../Data/Examples/fire_' ,year_col_name='YEAR_',
                year_sub_list=range(2008,2014))



c:\users\python3\documents\ts-raster\tsraster\prep.py:388: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  polys['ONES'] = 1


RasterioIOError: ../Data/Examples/aet-198403.tif: No such file or directory

In [ ]:
# run all individual years 2000 to 2018
shp_fn = r'../Data/Actual/Fires/Inputs/fire17_1.shp'

for year in range(1950,2019):
    poly_rasterizer_year_group(poly = shp_fn, 
                    raster_exmpl = r'../Data/Examples/3month/aet-198403.tif',
                    raster_path_prefix = r'../Data/Actual\Fires/Rasters/fire_' ,
                    year_col_name='YEAR_',
                    year_sub_list=year)



In [ ]:
# Gather multiple date ranges
year_groups = []
for f, b in zip(range(1951,2015,5), range(1955,2020,5)):
    year_groups.append([f, b])
    
print(year_groups)


In [ ]:
# run all 5 year date ranges from 1950 to 2018
shp_fn = r'../Data/Actual/Fires/Inputs/fire17_1.shp'

for year in year_groups:
    poly_rasterizer_year_group(poly = shp_fn, 
                    raster_exmpl = r'../Data/Examples/aet-198403.tif',
                    raster_path_prefix = r'../Data/Actual/Fires/Outputs/fire_' ,
                    year_col_name='YEAR_',
                    year_sub_list=year)

Move climate data into correct year_group folders

In [ ]:
import shutil
from pathlib import Path


def chk_mak_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        
def int_globber(ints):
    # create regular expression for specific year 
    return ''.join(["["+i+"]" for i in str(ints)])

def path_globber(a_dir,year_group,metric):
     # gets all in_path files in a date range 
     l = [glob(os.path.join(a_dir,metric+"-"+int_globber(i)+"*")) 
              for i in range(year_group[0],year_group[1])]
     return  [item for sublist in l for item in sublist]

    
in_dir = 'E:/HIST 1000m/'
out_dir = '../Data/Actual/Climate'

for metric in ['ppt','tmx','aet','cwd','pet','pck']:   
    for year_group in year_groups:
        # check for and make output dir
        year_group_dir = str(year_group[0])+"_"+ str(year_group[1])
        out_dir_metric = os.path.join(out_dir,year_group_dir,metric)
        chk_mak_dir(out_dir_metric)
        
        # make lists of files to move and destinations
        files_to_move = path_globber(in_dir, year_group,metric)
        files_out_move =[os.path.join(out_dir_metric, 
                                      os.path.basename(i)) for i in files_to_move]
        #print(pd.DataFrame([files_to_move,files_out_move])) # print examples
        
        # move files 
        for index, item in enumerate(files_to_move ):
            if Path(files_out_move[index]).is_file():
                next
            else:
                shutil.move(files_to_move[index], files_out_move[index])


In [ ]:
import pandas as pd
pd.DataFrame([files_to_move,files_out_move])



# Rasterize State Polygon

function to rasterize polygons (assigns 1s to features) and can provide buffer at multiples of example raster resolution.

In [ ]:
from tsraster import prep
import rasterio
from matplotlib import pyplot

In [ ]:
# rasterize state polygon using same parameters as raster_ex, add 10 cell buffer
prep.poly_rasterizer(poly = 'F:/Boundary/StatePoly.shp', raster_ex = r'F:/3month/aet-198401.tif',
                raster_path_prefix = r'../Data\Examples\buffer/StatePoly_buf',buffer_poly_cells=0)

In [ ]:
dataset = rasterio.open(r'../Data/Examples/TestOutputs/StatePoly_buf.tif')


pyplot.imshow(dataset.read(1), cmap='pink')
pyplot.show()